# Plan

- 







In [1]:
!pip install bpe

     |████████████████████████████████| 358kB 9.6MB/s 
     |████████████████████████████████| 21.0MB 38.4MB/s 
     |████████████████████████████████| 747kB 41.2MB/s 


In [23]:
# imports


from google.colab import files
import pandas as pd
import io
import tensorflow as tf
from keras.preprocessing.text import Tokenizer
import numpy as np
import string
from keras.utils import to_categorical
from bpe import Encoder
from collections import defaultdict

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from random import shuffle

In [3]:
# load data
uploaded = files.upload()

Saving babynames-clean.csv to babynames-clean.csv


In [4]:
names = pd.read_csv(io.BytesIO(uploaded['babynames-clean.csv']))
names.drop(names.columns[[1, 1]], axis=1, inplace=True) # drop genders

In [5]:
##########################
#         BPE            #
##########################

corpus = names["John"].tolist() # to list ify

encoder = Encoder(800, pct_bpe=1)  # encode it all
encoder.fit(corpus) # fit it to the corpus

names["tokenized_names"] = names.apply(lambda row:encoder.tokenize(row["John"]), axis=1) # make the collumn

names.head()


,John,tokenized_names
0,William,"[__sow, wi, ll, ia, m, __eow]"
1,James,"[__sow, ja, me, s, __eow]"
2,Charles,"[__sow, ch, ar, le, s, __eow]"
3,George,"[__sow, ge, or, ge, __eow]"
4,Frank,"[__sow, fr, an, k, __eow]"


In [6]:
## find max_len

# high = 0 # initialize high
# high = names.apply(lambda row: max(high, len(row["tokenized_names"])), axis=1) # jank, could cut the max func here
# max_len = max(high.tolist()) # take the highest len value
# print(max_len)


8


In [7]:
##########################
#       Padding          #
##########################

# names["padded"] = names.apply(lambda row: row["tokenized_names"]+(max_len - len(row["tokenized_names"]))*["__pad"], axis=1) # pad em

# names.head()


,John,tokenized_names,padded
0,William,"[__sow, wi, ll, ia, m, __eow]","[__sow, wi, ll, ia, m, __eow, __pad, __pad]"
1,James,"[__sow, ja, me, s, __eow]","[__sow, ja, me, s, __eow, __pad, __pad, __pad]"
2,Charles,"[__sow, ch, ar, le, s, __eow]","[__sow, ch, ar, le, s, __eow, __pad, __pad]"
3,George,"[__sow, ge, or, ge, __eow]","[__sow, ge, or, ge, __eow, __pad, __pad, __pad]"
4,Frank,"[__sow, fr, an, k, __eow]","[__sow, fr, an, k, __eow, __pad, __pad, __pad]"


In [7]:
##########################
#       Tokenize         #
##########################

tok = defaultdict(lambda: len(tok)) # define the default dict. add one to deal with new keys

names["toked"] = [[tok[j] for j in i] for i in names["tokenized_names"].tolist()] # tokenize it!

names.head()


,John,tokenized_names,toked
0,William,"[__sow, wi, ll, ia, m, __eow]","[0, 1, 2, 3, 4, 5]"
1,James,"[__sow, ja, me, s, __eow]","[0, 6, 7, 8, 5]"
2,Charles,"[__sow, ch, ar, le, s, __eow]","[0, 9, 10, 11, 8, 5]"
3,George,"[__sow, ge, or, ge, __eow]","[0, 12, 13, 12, 5]"
4,Frank,"[__sow, fr, an, k, __eow]","[0, 14, 15, 16, 5]"


In [8]:
##########################
#       One Hot          #
##########################

names["one_hot"] = names.apply(lambda row: to_categorical(row["toked"], num_classes=len(tok)), axis=1) # one hot it! num_classes is len of tok 

names.head()

,John,tokenized_names,toked,one_hot
0,William,"[__sow, wi, ll, ia, m, __eow]","[0, 1, 2, 3, 4, 5]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
1,James,"[__sow, ja, me, s, __eow]","[0, 6, 7, 8, 5]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
2,Charles,"[__sow, ch, ar, le, s, __eow]","[0, 9, 10, 11, 8, 5]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
3,George,"[__sow, ge, or, ge, __eow]","[0, 12, 13, 12, 5]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
4,Frank,"[__sow, fr, an, k, __eow]","[0, 14, 15, 16, 5]","[[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."


In [27]:
########################
#     Markhov Time     #
########################

data = names["one_hot"].tolist()

pairs = []
p = []

for i in data:
  for j in i:
    if len(p) == 2:
      pairs.append(p)
      p = []
    p.append(list(j))

shuffle(pairs)

print(pairs[:3])






[[[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.

In [11]:
#####################
#   df to tensor    #
#####################

# data = torch.from_numpy(np.array(names["one_hot"].tolist()))

# print(data)


-





In [29]:
###################################
#     Model :clap: Time :clap:    #
###################################


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # layers
        self.layer1 = nn.Linear(434, 256)
        self.layer2 = nn.Linear(256, 128)
        self.layer3 = nn.Linear(128, 256)
        self.layer4 = nn.Linear(256, 434)
        self.output_ac = nn.Softmax(dim=1)

    def forward(self, x):
        # model structure
        net = self.layer1(x)
        net = self.layer2(net)
        net = self.layer3(net)
        net = self.layer4(net)
        net = self.output_ac(net)

        return net


net = Net()





In [30]:
###################
#   OPTIMIZER     #
###################


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)





In [ ]:
###################
#     TRAINING    #
###################

for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        # inputs, labels = data
        name = 

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


